In [1]:
import pandas as pd

In [2]:
filings = pd.read_csv('data/nc_validation_filings (1).csv')
filings.head()

,ticker,accession_number
0,ACAD,0001564590-18-003526
1,ACC,0001283630-18-000024
2,AFSI,0001365555-18-000052
3,AHL,0001267395-18-000024
4,AKRX,0001628280-18-002518


In [ ]:
purchase = pd.read_csv('data/share_repurchase_paragraphs.csv')
purchase.head()